In [41]:
import cv2
import numpy as np

In [42]:
img = cv2.imread('./img/taekwonv1.jpg')
img_draw = img.copy()
# Create mask
mask = np.zeros(img.shape[:2], dtype=np.uint8)
# Initialize coordinates of a rectangular area
rect = [0, 0, 0, 0]
# Grabcut initial mode
mode = cv2.GC_EVAL

In [43]:
# Background and foreground model buffer
bgdmodel = np.zeros((1, 65), np.float64)
fgdmodel = np.zeros((1, 65), np.float64)

In [44]:
# Mouse event handling function
def onMouse(event, x, y, flags, param):
    global mouse_mode, rect, mask, mode
    if event == cv2.EVENT_LBUTTONDOWN:
        if flags <= 1:                      # If do not press any key
            mode = cv2.GC_INIT_WITH_RECT    # Starting drag, rectangle mode
            rect[:2] = x, y                 # Save starting coordinates
    elif event == cv2.EVENT_MOUSEMOVE and flags & cv2.EVENT_FLAG_LBUTTON:
        if mode == cv2.GC_INIT_WITH_RECT:   # Dragging in progress
            img_temp = img.copy()
            # Display a drag rectangle
            cv2.rectangle(img_temp, (rect[0], rect[1]), (x, y), (0, 255, 0), 2)
            cv2.imshow('img', img_temp)
        elif flags > 1:                             # Pressed state of a key
            mode = cv2.GC_INIT_WITH_MASK            # mask mode
            if flags & cv2.EVENT_FLAG_CTRLKEY:      # Ctrl key, definitely foreground
                # Display white dots on the screen
                cv2.circle(img_draw, (x, y), 3, (255, 255, 255), -1)
                # Fill the mask with GC_FGD
                cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)
            if flags & cv2.EVENT_FLAG_SHIFTKEY:     # Shift key, definitely background
                # Display black dots on the screen
                cv2.circle(img_draw, (x, y), 3, (0, 0, 0), -1)
                # Fill the mask with GC_BGD
                cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)
            cv2.imshow('img', img_draw)    # Display drawn image
    elif event == cv2.EVENT_LBUTTONUP:
        if mode == cv2.GC_INIT_WITH_RECT:
            rect[2:] = x, y            # Collect the last coordinate of a rectangle
            # Draw rectangle and dispay it
            cv2.rectangle(img_draw, (rect[0], rect[1]), (x, y), (255, 0, 0), 2)
            cv2.imshow('img', img_draw)
        # Apply grabcut
        cv2.grabCut(img, mask, tuple(rect), bgdmodel, fgdmodel, 1, mode)
        img2 = img.copy()
        # Fill the area marked as 'definitely background' and 'probably background' with 0 on the mask
        img2[(mask==cv2.GC_BGD) | (mask==cv2.GC_PR_BGD)] = 0
        cv2.imshow('grabcut', img2)    # Print final result
        mode = cv2.GC_EVAL             # Reset grabcut mode

In [45]:
# Display initial screen and register mouse events
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)
while True:
    if cv2.waitKey() & 0xFF == 27:
        break
cv2.destroyAllWindows()